# Capstone Project
## Image classifier for the SVHN dataset
### Instructions

In this notebook, you will create a neural network that classifies real-world images digits. You will use concepts from throughout this course in building, training, testing, validating and saving your Tensorflow classifier model.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [9]:
import logging
import matplotlib.pyplot as plt

import random
import tensorflow as tf
import time
from scipy.io import loadmat
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import datasets, models, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

In [1]:
import numpy as np
import tensorflow as tf

from tensorboard.plugins.hparams import api as hp
from absl import app, flags
import shutil



In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)
tf.config.experimental.set_virtual_device_configuration(
  gpus[0],
  [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
)

In [15]:
HP_LAYER_COUNT = hp.HParam("layer_count", hp.IntInterval(1, 3))
HP_NUM_UNITS  = hp.HParam("num_units", hp.Discrete([64, 128, 256]))
HP_LEARNING_RATE = hp.HParam("learning_rate", hp.RealInterval(0.0005, 0.0021))

METRIC_ACCURACY = 'accuracy'

!rm -rf logs
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_LAYER_COUNT, HP_NUM_UNITS, HP_LEARNING_RATE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )
    
print('-'*25, 'HP_LAYER_COUNT' ,'-'*25)    
for deep_layers in range(HP_LAYER_COUNT.domain.min_value,
                         HP_LAYER_COUNT.domain.max_value):
    print(deep_layers)
    
print('-'*25, 'HP_NUM_UNITS' ,'-'*25)    
for deep_layer_size in HP_NUM_UNITS.domain.values:
    print(deep_layer_size)
    
print('-'*25, 'HP_LEARNING_RATE' ,'-'*25)    
for learning_rate in np.arange(HP_LEARNING_RATE.domain.min_value,
                               HP_LEARNING_RATE.domain.max_value,
                               0.0005):
    print(learning_rate)
    


------------------------- HP_LAYER_COUNT -------------------------
1
2
------------------------- HP_NUM_UNITS -------------------------
64
128
256
------------------------- HP_LEARNING_RATE -------------------------
0.0005
0.001
0.0015
0.002


In [16]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images = train_images/255.0
test_images = test_images/255.0

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

train_val_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)) 
train_val_dataset = train_val_dataset.shuffle(train_images.shape[0])

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_size = int(0.85*train_images.shape[0])
train_dataset = train_val_dataset.take(train_size) 
val_dataset = train_val_dataset.skip(train_size)

batch_size = 32
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
val_dataset = val_dataset.batch(batch_size*10)
test_dataset = test_dataset.batch(batch_size*10)

input_shape = train_dataset.element_spec[0].shape[1:]

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [25]:
def train_test_model(hparams, epochs):
    model = models.Sequential()
    model.add(layers.Flatten()) 
        
    for _ in range(hparams[HP_LAYER_COUNT]):
        model.add(
            layers.Dense(
                units=hparams[HP_NUM_UNITS],
                activation="relu"
            ))    
    model.add(layers.Dense(10, activation='sigmoid')) 

    model.compile(optimizer=optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE]),
                  loss=losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"],
    )
    
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)  
    
    model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, 
              callbacks=[early_stopping])
    _, accuracy = model.evaluate(test_dataset)

    return accuracy

def run(run_dir, hparams, epochs):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, epochs)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1) 
        print('Test accuracy: {:0.1f}%'.format(accuracy*100))
    

def run_all(epochs=5):
    session_num = 0

    for num_units in HP_NUM_UNITS.domain.values:
        for lay_count in range(HP_LAYER_COUNT.domain.min_value, HP_LAYER_COUNT.domain.max_value):
            for learning_rate in np.arange(HP_LEARNING_RATE.domain.min_value, 
                                           HP_LEARNING_RATE.domain.max_value,
                                           0.0005):
                hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_LAYER_COUNT: lay_count,
                    HP_LEARNING_RATE: learning_rate,
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams, epochs)
                session_num += 1    

In [27]:
tf.keras.backend.set_floatx('float64')
run_all(epochs=10)

--- Starting trial: run-0
{'num_units': 64, 'layer_count': 1, 'learning_rate': 0.0005}
Epoch 1/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.4390 - accuracy: 0.8863 - val_loss: 0.2488 - val_accuracy: 0.9322
Epoch 2/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.2233 - accuracy: 0.9372 - val_loss: 0.2000 - val_accuracy: 0.9442
Epoch 3/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.1761 - accuracy: 0.9512 - val_loss: 0.1465 - val_accuracy: 0.9610
Epoch 4/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.1454 - accuracy: 0.9587 - val_loss: 0.1359 - val_accuracy: 0.9618
Epoch 5/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.1265 - accuracy: 0.9641 - val_loss: 0.1155 - val_accuracy: 0.9681
Epoch 6/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.1083 - accuracy: 0.9689 - val_loss: 0.0973 - val_accuracy: 0.9718
Epoch 7/10
1593/1593 [===========================

Epoch 1/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.3251 - accuracy: 0.9057 - val_loss: 0.1759 - val_accuracy: 0.9503
Epoch 2/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.1465 - accuracy: 0.9562 - val_loss: 0.0999 - val_accuracy: 0.9698
Epoch 3/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.1089 - accuracy: 0.9668 - val_loss: 0.0920 - val_accuracy: 0.9716
Epoch 4/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0853 - accuracy: 0.9744 - val_loss: 0.0731 - val_accuracy: 0.9772
Epoch 5/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0715 - accuracy: 0.9776 - val_loss: 0.0541 - val_accuracy: 0.9831
Epoch 6/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0598 - accuracy: 0.9805 - val_loss: 0.0390 - val_accuracy: 0.9877
Epoch 7/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0520 - accuracy: 0.9833 - val_loss: 0.0470 - val_accuracy:

1593/1593 [==============================] - 5s 3ms/step - loss: 0.2739 - accuracy: 0.9215 - val_loss: 0.1361 - val_accuracy: 0.9592
Epoch 2/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.1240 - accuracy: 0.9636 - val_loss: 0.0855 - val_accuracy: 0.9749
Epoch 3/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.0882 - accuracy: 0.9734 - val_loss: 0.0700 - val_accuracy: 0.9786
Epoch 4/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.0676 - accuracy: 0.9794 - val_loss: 0.0554 - val_accuracy: 0.9837
Epoch 5/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.0549 - accuracy: 0.9830 - val_loss: 0.0453 - val_accuracy: 0.9860
Epoch 6/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.0446 - accuracy: 0.9864 - val_loss: 0.0417 - val_accuracy: 0.9866
Epoch 7/10
1593/1593 [==============================] - 4s 3ms/step - loss: 0.0397 - accuracy: 0.9874 - val_loss: 0.0352 - val_accuracy: 0.9894
Epo

1593/1593 [==============================] - 5s 3ms/step - loss: 0.2352 - accuracy: 0.9290 - val_loss: 0.0997 - val_accuracy: 0.9681
Epoch 2/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.1059 - accuracy: 0.9673 - val_loss: 0.0878 - val_accuracy: 0.9719
Epoch 3/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0780 - accuracy: 0.9763 - val_loss: 0.0788 - val_accuracy: 0.9769
Epoch 4/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0672 - accuracy: 0.9786 - val_loss: 0.0586 - val_accuracy: 0.9816
Epoch 5/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0578 - accuracy: 0.9825 - val_loss: 0.0477 - val_accuracy: 0.9836
Epoch 6/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0487 - accuracy: 0.9841 - val_loss: 0.0352 - val_accuracy: 0.9886
Epoch 7/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0432 - accuracy: 0.9869 - val_loss: 0.0340 - val_accuracy: 0.9903
Epo

1593/1593 [==============================] - 5s 3ms/step - loss: 0.2742 - accuracy: 0.9221 - val_loss: 0.1331 - val_accuracy: 0.9613
Epoch 2/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.1120 - accuracy: 0.9661 - val_loss: 0.0868 - val_accuracy: 0.9724
Epoch 3/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0762 - accuracy: 0.9763 - val_loss: 0.0512 - val_accuracy: 0.9847
Epoch 4/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0567 - accuracy: 0.9820 - val_loss: 0.0426 - val_accuracy: 0.9874
Epoch 5/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0433 - accuracy: 0.9864 - val_loss: 0.0265 - val_accuracy: 0.9921
Epoch 6/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0354 - accuracy: 0.9885 - val_loss: 0.0266 - val_accuracy: 0.9918
Epoch 7/10
1593/1593 [==============================] - 5s 3ms/step - loss: 0.0286 - accuracy: 0.9909 - val_loss: 0.0216 - val_accuracy: 0.9919
Epo

In [ ]:
! tensorboard --logdir logs/hparam_tuning --bind_all

2020-12-09 10:14:28.244088: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorBoard 2.3.0 at http://b7fbff9a047a:6006/ (Press CTRL+C to quit)


打开 http://15.15.166.35:16006/